<a href="https://colab.research.google.com/github/Neil-Cardoz/Deep-Learning/blob/main/DLL_LAB_RNN_IMPLEMENTATION_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contractions -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.6 MB/s eta 0:00:00


In [ ]:
# NLP
import string, re, nltk
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import spacy
import contractions

In [ ]:
nltk.download("all")
!python -m spacy download en_core_web_sm

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 91.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Import Tensoflow libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Get Data

In [ ]:
!wget -c http://zenodo.org/records/3355823/files/ecommerceDataset.csv

URL transformed to HTTPS due to an HSTS policy
--2025-09-22 09:56:12--  https://zenodo.org/records/3355823/files/ecommerceDataset.csv
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.48.194, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36949114 (35M) [text/plain]
Saving to: ‘ecommerceDataset.csv’

ecommerceDataset.cs 100%[===================>]  35.24M  4.26MB/s    in 23s     

2025-09-22 09:56:36 (1.50 MB/s) - ‘ecommerceDataset.csv’ saved [36949114/36949114]



In [ ]:
import pandas as pd

dataset = pd.read_csv("ecommerceDataset.csv")
dataset.head()

,Household,"Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some for eternal bliss.so bring home this elegant print that is lushed with rich colors that makes it nothing but sheer elegance to be to your friends and family.it would be treasured forever by whoever your lucky recipient is. Liven up your place with these intriguing paintings that are high definition hd graphic digital prints for home, office or any room."
0,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
1,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
2,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
3,Household,Incredible Gifts India Wooden Happy Birthday U...
4,Household,Pitaara Box Romantic Venice Canvas Painting 6m...


# Text Cleaning

In [ ]:
regex = RegexpTokenizer("\[\w']+")

In [ ]:
# Lowercse
def text_lower(text):
  text = text.lower()
  return text

In [ ]:
# Remove Whitespace
def remove_whitespace(text):
  text = text.strip()
  return text

In [ ]:
# Remove Puntutation
def remove_punctuation(text):
  punct = string.punctuation
  punct = punct.replace("-", "")
  text = text.translate(str.maketrans("", "", punct))
  return text

In [ ]:
# Remove HTML
def remove_html(text):
  html = re.compile("<.*?>")
  text = html.sub(r"", text)
  return text

In [ ]:
import re

# Removing Emojis
def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols

        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)


In [ ]:
# Remove URLs
def remove_urls(text):
  text = re.sub(r"http://\S+|https://\S+", "", text)
  return text

In [ ]:
# Convert Abbreviated words
abbrex = pd.read_csv("abbreviations.csv", names=['SMF','FF'])
abbrex.head()

In [ ]:
abbrev_lower = pd.Dataframe()
abbrev_lower['SMF'] = abbrex['SMF'].apply(text_lower)
abbrev_lower['FF'] = abbrex['FF'].apply(text_lower)
abbrev_dict = dict(zip(abbrev_lower.SMF, abbrev_lower.FF))
abrev_words = list(abbrev_dict.keys())

In [ ]:
def convert_abbrev(word):
  words= []
  for word in regexp.tokenize(text):
    if word in abbrex.words:

      else:
        words = words + word.split()

    text_converted = " ".join(words)

    return text_converted

SyntaxError: expected ':' (ipython-input-2906560001.py, line 3)

In [ ]:
def covnertcontractions

In [ ]:
def remove_stopwords(text):
  text = " ".join([word for word in text.split() if word not in stopwords.words("english")])
  return text

In [ ]:
# Lemmatization
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def lemmatization(text):
  text = nlp(text)
  text = " ".join([word.lemma_ if word.lemma_ != "-PRON-" else word.text for word in text])
  return

In [ ]:
def discard_non_alpha(text):
  word_list_non_aplha = [word for word in regexp.tokenize(text) if not word.isalpha()]
  text = " ".join(word_list_non_aplha)
  return text

In [ ]:
# Aggrigating all the definitions
def text_cleaning(text):
  text = text_lower(text)
  text = remove_whitespace(text)
  text = re.sub('\n', '',text)
  text = re.sub('\[.?*\]', '', text)
  text = remove_http_links (text)
  text = remove_punctuation(text)
  text = remove_html(text)
  text = remove_emoji(text)
  text = convert_abbrev(text)
  text = remove_contractions(text)
  text = remove_stopwords(text)
  text = remove_stopwords(text)
  text = lemmatization(text)
  text = discard_non_alpha(text)
  return text

In [ ]:
# Apply on Dataset
data['Description_Clean'] = data['Description'].apply(text_cleaning)
data.head()

Label Encoding


In [ ]:
from operator import inv
le = LabelEncoder()
data['Encoded_Labels'] = le.fit_transform(data['Labels'])
#label = data['Encoded_Labels'].unique()
inv_label = le.inverse_transform(data['Encoded_Labels'])
data.head()